In [16]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import xgboost as xgb
from sklearn.model_selection import train_test_split
import seaborn as sns 
import matplotlib.pyplot as plt
#from sklearn.preprocessing import OneHotEncoder

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3331, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-16-dd16ae328b0d>", line 9, in <module>
    import seaborn as sns
  File "/opt/conda/lib/python3.7/site-packages/seaborn/__init__.py", line 2, in <module>
    import matplotlib as mpl
  File "/opt/conda/lib/python3.7/site-packages/matplotlib/__init__.py", line 1013, in <module>
    rcParams = rc_params()
  File "/opt/conda/lib/python3.7/site-packages/matplotlib/__init__.py", line 880, in rc_params
    return rc_params_from_file(matplotlib_fname(), fail_on_error)
  File "/opt/conda/lib/python3.7/site-packages/matplotlib/__init__.py", line 697, in matplotlib_fname
    for fname in gen_candidates():
  File "/opt/conda/lib/python3.7/site-packages/matplotlib/__init__.py", line 686, in gen_candidates
    yield os.path.join(os.getcwd(), 'matplotlibrc')
FileNotFoundError: [Errno 2]

FileNotFoundError: [Errno 2] No such file or directory

In [ ]:
df = pd.read_csv('/input/titanic/train.csv', index_col=0)
df['Type']='Train'
df.head(3)

In [ ]:
dfval = pd.read_csv('/input/titanic/test.csv', index_col=0)
dfval['Type']='Test'
dfval['Survived']=''
dfval.head(3)

In [ ]:
#dfx = pd.concat([df, dft])
dfx = df

In [ ]:
dfx.isna().sum()

In [ ]:
for col in list(dfx):
    if ((dfx[col].dtype =='object') and (dfx[col].name !='Name')):
        print(dfx.groupby([col])['Survived'].count().sort_values(ascending=False).head(10))

In [ ]:
#Cabin can be dropped, value is missing for all PClass
df[df['Cabin'].isna()]#['Pclass'].value_counts()

In [ ]:
df[df['Embarked'].isna()]

In [ ]:
df[['Embarked']].value_counts()

In [ ]:
#embarked can be dropped, its port of entry
df[df['Embarked']=='C'].sample(10)

In [ ]:
# Age distribution
sns.histplot(x = 'Age', data = df, binwidth = 5, kde = True)
plt.title('Age Distribution')
plt.show()

# Fare distribution
sns.histplot(x = 'Fare', data = df, binwidth = 25, kde = True)
plt.title('Fare Distribution')
plt.show()

In [ ]:
df[df['Age'].isna()]

In [ ]:
#no outliers in age
df[df['Age']>70]

In [ ]:
#age missing for 20% date, assign mean or 0 (new col flag)
df['Age'].mean()

In [ ]:
#fare combined for some cabins and may be based on port of entry, may influence incorrect results, will try
df[df['Fare']>200]

In [ ]:
def transform_df(dfx):
    tdf = dfx[['Survived','Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare' , 'Embarked', 'Type']]
    tdf['Age_Exists'] = tdf['Age'].isnull()
    tdf['Age_Exists'] = tdf['Age_Exists'].apply(lambda x: 0 if x is True else 1)
    #tdf['Age'].fillna(df['Age'].mean(), inplace=True)
    tdf['Age'].fillna(0, inplace=True)
    s=tdf['Embarked'].mode()
    tdf['Embarked'].fillna(s[0], inplace=True)
    tdf['Fare'].fillna(0, inplace=True)
    return tdf


In [ ]:

tdf=transform_df(tdf)
tdf.sample(5)

In [ ]:
tdf.isna().sum()

In [ ]:
x = tdf[tdf['Type']=='Train'][['Pclass', 'Sex', 'Age', 'SibSp', 'Parch']] #, 'Fare' , 'Embarked', 'Age_Exists']]
y = tdf[tdf['Type']=='Train'][['Survived']]
x = pd.get_dummies(x, drop_first=True)


In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.001, random_state=42)


In [ ]:
#Train the XGboost Model for Classification
model1 = xgb.XGBClassifier()
model2 = xgb.XGBClassifier(n_estimators=100, max_depth=8, learning_rate=0.1, subsample=0.5)

train_model1 = model1.fit(x_train, y_train)
train_model2 = model2.fit(x_train, y_train)

In [ ]:
#prediction and Classification Report
from sklearn.metrics import classification_report

pred1 = train_model1.predict(x_test)
pred2 = train_model2.predict(x_test)



In [ ]:
print('Model 1 XGboost Report:')
r1=classification_report(y_test, pred1)
print(r1)
r2=classification_report(y_test, pred2)
print(r2)

In [ ]:
model3 = xgb.XGBClassifier(n_estimators=80, max_depth=8, learning_rate=0.05, subsample=0.5, gamma=0, min_child_weight=2, objective= 'binary:logistic')

train_model3 = model3.fit(x_train, y_train)
pred3 = train_model3.predict(x_test)
print(classification_report(y_test, pred3))

In [ ]:
dfsub = pd.read_csv('/kaggle/input/titanic/gender_submission.csv', index_col=0)
dfsub

In [ ]:
#prediction using the model with test data
dfval=transform_df(dfval)
dfval.sample(5)
xv=dfval[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch']] 
x_val = pd.get_dummies(xv, drop_first=True)
pred_sub = train_model3.predict(x_val)


In [ ]:
len(pred_sub)
dfsub['Survived']=pred_sub
dfsub.to_csv('submission.csv') #, index=False)

In [ ]:
from sklearn.model_selection import GridSearchCV

param_test = {
 'max_depth':[4,5,6],
 'min_child_weight':[4,5,6]
}
gsearch = GridSearchCV(estimator = xgb.XGBClassifier( learning_rate=0.1, n_estimators=120, max_depth=5,
 min_child_weight=2, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
 param_grid = param_test, scoring='roc_auc',n_jobs=4,iid=False, cv=5)

#train_model4 = gsearch.fit(x_train, y_train)
#pred4 = train_model4.predict(x_test)


In [ ]:
print(classification_report(y_test, pred4))
train_model4.cv_results_

In [ ]:
from sklearn import svm
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

model_params = {
    'svm': {
        'model': svm.SVR(gamma='auto'),
        'params' : {
            'C': [10,20, 50],
            'kernel': ['rbf','linear']
        }  
    },
    'random_forest': {
        'model': RandomForestRegressor(),
        'params' : {
            'n_estimators': [50, 100]
        }
    }
}


In [ ]:
scores = []

for model_name, mp in model_params.items():
    clf =  GridSearchCV(mp['model'], mp['params'], cv=5, return_train_score=True)
    #clf.fit(x_train, y_train)
    scores.append({
        'model': model_name,
        'best_score': clf.best_score_,
        'best_params': clf.best_params_
    })
    
dfs = pd.DataFrame(scores,columns=['model','best_score','best_params'])
dfs

In [ ]:
df2=pd.read_csv('/kaggle/input/titanic/gender_submission.csv')#, Index=False)
df2

In [ ]:
df2.to_csv('submission.csv') #, index=False)